In [518]:
import numpy as np
import random as rd
from sys import getsizeof
import math 

### State

In [519]:
def visualize_board(state):
    n = len(state)
    columns = "abcdefgh"  
    
    print("   " + "  ".join(columns[:n]))

    for row in range(n):
        line = f"{row + 1}  " 
        for col in range(n):
            if state[col] == row + 1: 
                line += "Q  "
            else:
                line += ".  "
        line += f"{row + 1}"  
        print(line)
    
    print("   " + "  ".join(columns[:n]))

# Example usage:
state = np.array([1,2,3,4,5,6,7,8])
visualize_board(state)

   a  b  c  d  e  f  g  h
1  Q  .  .  .  .  .  .  .  1
2  .  Q  .  .  .  .  .  .  2
3  .  .  Q  .  .  .  .  .  3
4  .  .  .  Q  .  .  .  .  4
5  .  .  .  .  Q  .  .  .  5
6  .  .  .  .  .  Q  .  .  6
7  .  .  .  .  .  .  Q  .  7
8  .  .  .  .  .  .  .  Q  8
   a  b  c  d  e  f  g  h


### Fitness Function

In [520]:
def fitness_function(state):
    # To Check: 
    # 1] Queens can't be in the same row (due to state representation, no need to check for conflicting columns)
    # 2] Queens can't be in the same diagonal
    # Fitness Functions returns: '#(Conflicting Pairs of Queens)'
    n = len(state)
    conflict_counter = 0
    for Q1 in range(n):
        for Q2 in range(Q1+1, n):
            # counts conflicts row-wise
            if(state[Q1] == state[Q2]):
                conflict_counter += 1
            # counts conflicts diag-wise
            # EX: [_,1,_,_,_,_,6,_] -> (2,1) & (7,6) => DIAG: (2-7)=(1-6) <=> (-5)=(-5)
            if(state[Q1]-state[Q2] == Q1-Q2):
                conflict_counter += 1
    return conflict_counter

In [521]:
fitness_function(state)

28

### Population Generation

In [522]:
def init_population_generation(n):
    # low while is included while high value is excluded in the calculations
    return np.random.randint(low=1, high=8+1, size=(n,8))

In [523]:
population_size = 38 # choose an even number
state = init_population_generation(population_size)
# state

### Evaluate Function Fitness

In [524]:
def eval_curr_fitness(state, population_size):
    fitness_evals = []
    for i in range(0, population_size):
        fitness_evals.append(1 - fitness_function(state[i]) / 28)
    return np.array(fitness_evals)

fitness_evals = eval_curr_fitness(state, population_size)

In [525]:
fitness_evals

array([0.85714286, 0.67857143, 0.85714286, 0.82142857, 0.85714286,
       0.85714286, 0.92857143, 0.78571429, 0.92857143, 0.85714286,
       0.75      , 0.89285714, 0.85714286, 0.89285714, 0.82142857,
       0.71428571, 0.85714286, 0.85714286, 0.78571429, 0.75      ,
       0.67857143, 0.82142857, 0.75      , 0.82142857, 0.85714286,
       0.75      , 0.85714286, 0.78571429, 0.67857143, 0.85714286,
       0.78571429, 0.75      , 0.82142857, 0.82142857, 0.89285714,
       0.85714286, 0.85714286, 0.78571429])

### Selection

In [526]:
def selection(state, fitness_evals, population_size):
    selected_individuals = []
    copy_state = state
    copy_fitness_evals = fitness_evals
    while(len(selected_individuals) < population_size):
        if (copy_state.shape[0] == 0):
            copy_state = state
            copy_fitness_evals = fitness_evals
        i = np.random.randint(low=0, high=len(copy_state))
        if rd.random() < copy_fitness_evals[i]:
            selected_individuals.append(copy_state[i])
            copy_state = np.delete(arr=copy_state, obj=i, axis=0)
            copy_fitness_evals = np.delete(arr=copy_fitness_evals, obj=i, axis=0)
        
    return np.array(selected_individuals)

In [527]:
selected_individuals = selection(state, fitness_evals, population_size)

In [528]:
print(len(selected_individuals), population_size)

38 38


In [529]:
print(selected_individuals)

[[7 5 8 8 4 8 2 8]
 [7 4 3 3 2 2 7 1]
 [2 6 6 7 7 1 4 6]
 [3 1 8 7 8 7 1 4]
 [5 4 1 3 8 3 5 7]
 [8 8 6 4 3 7 1 8]
 [2 3 3 6 8 7 2 4]
 [4 2 1 7 5 2 8 1]
 [1 7 3 7 8 8 5 4]
 [7 3 5 6 5 4 8 4]
 [2 8 4 7 6 6 6 8]
 [2 4 3 1 7 1 3 4]
 [1 5 1 2 2 8 4 5]
 [3 3 4 3 8 4 3 6]
 [4 3 5 7 3 5 8 2]
 [4 5 4 3 1 6 2 6]
 [6 4 6 4 1 2 7 5]
 [4 5 3 1 3 4 8 8]
 [2 8 1 4 6 1 6 3]
 [1 1 7 2 2 5 1 3]
 [7 2 3 5 1 6 5 7]
 [5 2 8 3 7 1 5 7]
 [4 5 7 6 3 1 8 8]
 [2 7 1 4 4 8 4 1]
 [5 3 7 2 1 2 3 6]
 [8 1 2 7 5 6 4 7]
 [7 3 7 3 8 2 4 3]
 [5 3 8 6 7 5 8 8]
 [5 6 1 4 8 2 1 3]
 [1 2 5 1 6 3 3 1]
 [1 8 8 3 1 3 5 2]
 [5 4 7 4 5 4 3 6]
 [6 6 8 8 7 4 6 5]
 [8 5 7 3 4 4 4 4]
 [3 6 7 2 7 1 4 4]
 [4 7 5 4 1 2 4 4]
 [6 4 6 5 3 7 8 6]
 [2 2 6 6 8 2 8 6]]


### Crossover (Recombination)

In [541]:
# state = current boards after selection
# probability = probability of mutation, usually between 70-90%
def recombination(selected_individuals, population_size, crossover_rate):
    new_population = []
    copy_selected_individuals = selected_individuals.copy()

    while len(new_population) < population_size:
        crossover_point = np.random.randint(3, 6)
        dad_idx = np.random.randint(0, len(copy_selected_individuals))
        mom_idx = dad_idx
        while(dad_idx == mom_idx):
            mom_idx = np.random.randint(0, len(copy_selected_individuals))
        dad = copy_selected_individuals[dad_idx]
        mom = copy_selected_individuals[mom_idx]

        if(np.random.rand() < crossover_rate):
            child_one = child_two = [0] * 8
            for j in range(8):
                if(j < crossover_point):
                    child_one[j] = dad[j]
                    child_two[j] = mom[j]
                else:
                    child_one[j] = mom[j]
                    child_two[j] = dad[j]
            new_population.append(child_one)
            new_population.append(child_two)
        else:
            new_population.append(dad)
            new_population.append(mom)

    return np.array(new_population[:population_size])

In [531]:
print('BEFORE:', np.average(eval_curr_fitness(state, population_size)))
crossover_rate = 0.80
print('AFTER:', np.average(eval_curr_fitness(recombination(selected_individuals, population_size, crossover_rate), population_size)))

BEFORE: 0.8167293233082707
AFTER: 0.81203007518797


In [532]:
arr_100 = []
for _ in range(100):
    arr_100.append(np.average(eval_curr_fitness(recombination(selected_individuals, population_size, crossover_rate), population_size)))
print(np.average(arr_100))

0.8152255639097747


In [534]:
state = recombination(selected_individuals, population_size, crossover_rate)

In [535]:
state.shape

(38, 8)

### Mutation

In [538]:
# state = current boards after recombination
# mutation_prob = probability of mutation, usually between 1-5% (give it as a decimal, i.e. 0.03)
def mutation(state, mutation_prob):
    # experiment with different mutation strategies (single-gene mutations, swap mutations, or mix of both)
    for board in state:
        if(np.random.rand() < mutation_prob):
            new_gene = np.random.randint(1,8+1)
            insert_idx = np.random.randint(0,8)
            board[insert_idx] = new_gene
    return state

In [539]:
mutation_prob = 0.03
state = mutation(state, mutation_prob)

In [556]:
if(np.any(np.array([0.5, 0.3, 1]) == 1)):
    index = np.argmax(np.array([0.5, 0.3, 1]))
    print(index)

2
